In [1]:
%%capture
!pip install azure-storage-blob

!pip install boto3

In [2]:
%%capture
!pip install tensorflow

!pip install opencv-python

!pip install imutils

!pip install Pillow

!pip install scikit-learn

!pip install matplotlib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive/Data Science/Projects/Banking e-KYC/Codes'

/content/drive/MyDrive/Data Science/Projects/Banking e-KYC/Codes


In [5]:
%ls

aws_access.py                        Image_Similarity_Modeling_TransferLearning
azure_blob_access.py                 Image_Similary_Modeling.ipynb
complete_image_data_with_labels.csv  output/
config.py                            __pycache__/
Data_Exploration.ipynb               siamese_network.py
examples/                            utils.py
face_recognition.json


## **Loading Json file containing ID and Image path from Azure Blob Storage:**

In [6]:
import os, uuid
import json
import azure_blob_access
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [7]:
# Getting the connection string
connect_str = azure_blob_access.AZURE_STORAGE_CONNECTION_STRING

In [8]:
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [9]:
# Get the container object
container_name = "face-recognition"
container_client = blob_service_client.get_container_client(container= container_name) 

In [10]:
# Explore the contents of the container
print("\nListing blobs...")

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)


Listing blobs...
	face_recognition.json


In [11]:
# Get the blob client and download the json file
fileName = "face_recognition.json"
blob_client = container_client.get_blob_client(fileName)
streamdownloader = blob_client.download_blob()

 
json_obj = json.loads(streamdownloader.readall())
# print(json_obj)

json_obj = json.dumps(json_obj)

jsonfile = "face_recognition.json"
with open(jsonfile, 'w') as outfile:
    outfile.write(json_obj)

In [12]:
# Loading the json file 
with open("face_recognition.json", 'r') as f:
    face_recog = json.load(f)

print(face_recog)

{'ID': {'0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '10': 11, '11': 12, '12': 13, '13': 14, '14': 15, '15': 16, '16': 17, '17': 18, '18': 19, '19': 20, '20': 21, '21': 22, '22': 23, '23': 24, '24': 25, '25': 26, '26': 27, '27': 28, '28': 29, '29': 30, '30': 31, '31': 32, '32': 33, '33': 34, '34': 35, '35': 36, '36': 37, '37': 38, '38': 39, '39': 40, '40': 41, '41': 42, '42': 43, '43': 44, '44': 45, '45': 46, '46': 47, '47': 48, '48': 49, '49': 50}, 'Folder_link': {'0': 's3://face-recognition579/Data/1/', '1': 's3://face-recognition579/Data/2/', '2': 's3://face-recognition579/Data/3/', '3': 's3://face-recognition579/Data/4/', '4': 's3://face-recognition579/Data/5/', '5': 's3://face-recognition579/Data/6/', '6': 's3://face-recognition579/Data/7/', '7': 's3://face-recognition579/Data/8/', '8': 's3://face-recognition579/Data/9/', '9': 's3://face-recognition579/Data/10/', '10': 's3://face-recognition579/Data/11/', '11': 's3://face-recognition579/Data/12/

In [13]:
# Checking the keys in the dictionary
face_recog.keys()

dict_keys(['ID', 'Folder_link'])

## **Creating connection to AWS S3 to retrieve the image paths:**

In [14]:
import boto3
import aws_access
s3_client = boto3.client('s3', aws_access_key_id = aws_access.aws_access_key_id, 
                                aws_secret_access_key = aws_access.aws_secret_access_key)

In [17]:
# s3_client.list_objects(Bucket=bucket,Prefix=prefix)

In [15]:
# List objects in the bucket (s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/)
bucket = "hariharan-project-data"
prefix = "Banking-and-Finance/Face-Recognition-KYC-Tool/Data"

fileList = []
for obj in s3_client.list_objects(Bucket=bucket,Prefix=prefix)['Contents']:
    files = obj['Key']
    fileList.append("s3://hariharan-project-data/"+files)


In [18]:
print("Number of Images: ",len(fileList))
fileList[:10]

Number of Images:  212


['s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/1/1_1.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/1/1_2.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_1.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_2.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_3.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_4.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/11/11_1.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/11/11_2.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/12/12_1.jpg',
 's3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/12/12_2.jpg']

## **Creating a DataFrame with image paths and Ids:**

In [19]:
import pandas as pd 
pd.set_option('max_colwidth',None)

data = pd.DataFrame(fileList, columns=['s3_img_path'])
print(data.shape)
data.head()

(212, 1)


,s3_img_path
0,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/1/1_1.jpg
1,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/1/1_2.jpg
2,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_1.jpg
3,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_2.jpg
4,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_3.jpg


In [23]:
# data['s3_img_path'][4].split('/') #[6]

In [24]:
data['s3_img_path'][6].split('/')[6]
data['user_id'] = data['s3_img_path'].apply(lambda x: x.split('/')[6])

In [25]:
data.head()

,s3_img_path,user_id
0,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/1/1_1.jpg,1
1,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/1/1_2.jpg,1
2,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_1.jpg,10
3,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_2.jpg,10
4,s3://hariharan-project-data/Banking-and-Finance/Face-Recognition-KYC-Tool/Data/10/10_3.jpg,10


In [26]:
# EDA
print("Total Unique Users in the data: ", data.user_id.nunique())

median_img_cnt = data.groupby('user_id').count()['s3_img_path'].median()
print("Median number of images per user: ", median_img_cnt)

Total Unique Users in the data:  52
Median number of images per user:  3.0


In [ ]:
# saving the dataframe 
# data.to_csv("complete_image_data_with_labels.csv",index=False)

## **Setting up the Deep Learning Env:**

!pip install tensorflow

!pip install opencv-python

!pip install imutils

!pip install Pillow

!pip install scikit-learn

In [ ]:
import tensorflow as tf
tf.__version__

'2.9.2'

In [ ]:
from tensorflow import keras
keras.__version__ 

'2.9.0'

In [ ]:
import cv2 
cv2.__version__

'4.6.0'